<a href="https://colab.research.google.com/github/JBlizzard-sketch/LoanIQ/blob/main/Copy_of_LoanIQUpdated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------- Cell 1: Dependencies & Minimal Setup (patched with Faker) --------
# Installs pinned versions and includes faker (safe: no reinstalls if already present)

%%bash
set -euo pipefail

python -m pip install --quiet --upgrade \
  "numpy==1.26.4" \
  "scikit-learn==1.3.2" \
  "xgboost==1.7.6" \
  "shap==0.41.0" \
  "pyngrok==5.2.1" \
  "streamlit" \
  "pandas" \
  "imbalanced-learn" \
  "faker" \
  "joblib" \
  "sqlalchemy" \
  "openpyxl" \
  "python-dotenv" \
  || true

# --- Create persistent folder structure ---
mkdir -p /content/loan_app/modules/synth \
         /content/loan_app/modules/ml \
         /content/loan_app/modules/streamlit_app \
         /content/loan_app/data \
         /content/loan_app/models \
         /content/loan_app/logs

# --- Write ngrok authtoken ---
NGROK_TOKEN="31rYvgklL0EdX9bGLvTXc313efE_2GyDFGPUNAyFgB83bikTF"
mkdir -p ~/.ngrok2
cat > ~/.ngrok2/ngrok.yml <<NGY
authtoken: ${NGROK_TOKEN}
NGY
export NGROK_AUTHTOKEN="${NGROK_TOKEN}"

# --- Quick version check ---
python - <<'PY'
import importlib
pkgs = ["numpy","sklearn","xgboost","shap","pandas","streamlit","pyngrok","imbalanced_learn","faker"]
for p in pkgs:
    try:
        m = importlib.import_module(p)
        v = getattr(m, "__version__", None)
        print(f"{p}: {v}")
    except Exception as e:
        print(f"{p}: IMPORT ERROR -> {e}")
PY

echo "✅ Dependencies pinned. ✅ Faker included. ✅ Repo folders created. ✅ Ngrok token set."

Process is terminated.


In [ ]:

# -------- Cell 2/9 (CLEAN & COMPRESSED): Write Repo Modules --------
import os, sys
from pathlib import Path
import importlib

BASE = Path("/content/loan_app")
MODULES = BASE / "modules"
SYNTH_DIR = MODULES / "synth"
ML_DIR = MODULES / "ml"
STREAMLIT_DIR = MODULES / "streamlit_app"
DATA_DIR, MODELS_DIR, LOGS_DIR = BASE/"data", BASE/"models", BASE/"logs"

# Create folders
for d in [BASE, MODULES, SYNTH_DIR, ML_DIR, STREAMLIT_DIR, DATA_DIR, MODELS_DIR, LOGS_DIR]:
    d.mkdir(parents=True, exist_ok=True)
if str(BASE) not in sys.path:
    sys.path.insert(0, str(BASE))

def write_module(path: Path, code: str):
    path.write_text(code.strip()+"\n")
    # reopen sanity check
    with open(path) as f: f.read()

# -------- 1) auth.py --------
auth_py = r"""
import sqlite3, hashlib, datetime, os
DB_PATH = os.path.join('/content/loan_app','data','users.db')

def init_db():
    os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute('CREATE TABLE IF NOT EXISTS users (username TEXT PRIMARY KEY, password TEXT, role TEXT, created_at TEXT)')
    conn.commit(); conn.close()
    register_user('Admin','Shady868','admin')

def hash_pw(pw): return hashlib.sha256(pw.encode()).hexdigest()

def register_user(username, password, role='company'):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute('INSERT OR IGNORE INTO users VALUES (?,?,?,?)',
        (username, hash_pw(password), role, datetime.datetime.utcnow().isoformat()))
    conn.commit(); conn.close()

def authenticate(username, password):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute('SELECT password, role FROM users WHERE username=?',(username,))
    row = cur.fetchone(); conn.close()
    return (row and row[0]==hash_pw(password), row[1] if row else None)

def list_users():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute('SELECT username, role, created_at FROM users')
    rows = cur.fetchall(); conn.close(); return rows
"""
write_module(MODULES/"auth.py", auth_py)

# -------- 2) schema.py (compressed but rich) --------
schema_py = r"""
import re, pandas as pd, numpy as np, random
from datetime import datetime

SYNONYMS = {
  "client_id":["customer_id","cust_id","id"],
  "national_id":["id_no","idnumber","reg_no","reg_number"],
  "loan_amount":["amount","principal"],
  "branch":["office","location"],
  "product":["loan_product"],
  "loan_status":["status","performance","default","outcome"],
  "created_date":["created_at","disbursed_date","date"],
  "income":["monthly_income","salary"],
  "name":["customer_name","full_name"]
}
MALE, FEMALE = {"John","Peter","James","Joseph","Michael","David"}, {"Mary","Ann","Jane","Grace","Lucy","Sarah"}
TERM_OVERRIDES = {"inuka":4,"fadhili":6,"kuza":12,"agriadvance":16,"flexiloan":8,"bizboost":10}

def normalize_columns(df):
    df=df.copy()
    df.columns=[re.sub(r'[^\w]','_',c.strip().lower()) for c in df.columns]
    for canon,vars in SYNONYMS.items():
        for v in vars:
            if v in df.columns and canon not in df.columns: df.rename(columns={v:canon},inplace=True)
    return df

def parse_money(x):
    if pd.isna(x): return np.nan
    if isinstance(x,(int,float)): return float(x)
    s=str(x).lower().replace('kes','').replace('ksh','').replace(',','').strip()
    if s.endswith('k'): return float(s[:-1])*1000
    try: return float(re.sub(r'[^\d.]','',s))
    except: return np.nan

def parse_date(x):
    if pd.isna(x): return None
    s=str(x).strip()
    for fmt in ("%Y-%m-%d","%d/%m/%Y","%m/%d/%Y"):
        try: return pd.to_datetime(datetime.strptime(s,fmt))
        except: pass
    return pd.to_datetime(s, errors='coerce')

def estimate_age_from_id(nid):
    try:
        digits=int(str(nid)[:2])
        if 32<=digits<=34: return random.randint(27,30)
        if 29<=digits<=31: return random.randint(30,40)
        if 35<=digits<=38: return random.randint(18,25)
        return random.randint(25,50)
    except: return random.randint(25,50)

def gender_from_name(name):
    if not isinstance(name,str): return 'unknown'
    f=name.split()[0].capitalize()
    if f in MALE: return 'male'
    if f in FEMALE: return 'female'
    return 'female' if f.endswith(('a','e')) else 'male'

def term_weeks(prod):
    if not isinstance(prod,str): return None
    m=re.search(r'(\d+)\s*week',prod.lower())
    if m: return int(m.group(1))
    for k,v in TERM_OVERRIDES.items():
        if k in prod.lower(): return v
    return None

def coerce_and_enrich(df):
    df=normalize_columns(df)
    if "national_id" in df: df["unique_client_id"]=df["national_id"].astype(str)
    elif "client_id" in df: df["unique_client_id"]=df["client_id"].astype(str)
    else: df["unique_client_id"]=df.get("name","").astype(str)
    if "created_date" in df: df["created_date_parsed"]=df["created_date"].apply(parse_date)
    else: df["created_date_parsed"]=pd.to_datetime("today")
    df["loan_amount_num"]=df.get("loan_amount",np.nan).apply(parse_money)
    df["income_num"]=df.get("income",np.nan).apply(parse_money)
    df["product_term_weeks"]=df.get("product").apply(term_weeks) if "product" in df else None
    df["installment_size"]=df.apply(lambda r:(r["loan_amount_num"]/r["product_term_weeks"]) if pd.notna(r["loan_amount_num"]) and r.get("product_term_weeks") else np.nan,axis=1)
    df["gender_est"]=df.get("name","").apply(gender_from_name) if "name" in df else "unknown"
    df["age_est"]=df.get("national_id").apply(estimate_age_from_id) if "national_id" in df else None
    df["loan_to_income"]=df.apply(lambda r:(r["loan_amount_num"]/r["income_num"]) if r.get("income_num",0)>0 else np.nan,axis=1)
    df["is_young_borrower"]=df["age_est"].apply(lambda x: x<25 if pd.notna(x) else False)
    df["high_risk_amount"]=df["loan_amount_num"].apply(lambda x: x>300000 if pd.notna(x) else False)
    if "loan_status" in df:
        df["is_default"]=df["loan_status"].astype(str).str.lower().str.contains("default").astype(int)
    else: df["is_default"]=0
    df["days_since_issue"]=(pd.to_datetime("today")-pd.to_datetime(df["created_date_parsed"],errors="coerce")).dt.days.fillna(0).astype(int)
    return df

def prepare_for_ml(df, target="is_default", aggregate_by_client=False):
    df_en=coerce_and_enrich(df)
    num=["loan_amount_num","income_num","product_term_weeks","installment_size","loan_to_income","age_est","days_since_issue"]
    cat=[c for c in ["branch","product","town","gender_est"] if c in df_en.columns]
    X=df_en[num+cat].copy(); X[num]=X[num].fillna(0)
    if cat: X=pd.get_dummies(X,columns=cat,drop_first=True)
    y=df_en[target].astype(int)
    if aggregate_by_client:
        agg=X.groupby(df_en["unique_client_id"]).agg(["mean","max","min","sum","count"])
        agg.columns=["__".join(col) for col in agg.columns]
        y=y.groupby(df_en["unique_client_id"]).max()
        return agg,y,df_en
    return X,y,df_en
"""
write_module(MODULES/"schema.py", schema_py)

# -------- 3) synth/generator.py --------
synth_py = r"""
import random, pandas as pd
from faker import Faker
from datetime import datetime, timedelta

faker=Faker(); KENYAN_TOWNS=["Nairobi","Mombasa","Kisumu","Nakuru","Eldoret"]
BRANCHES=["Nairobi Branch","Mombasa Branch","Kisumu Branch","Nakuru Branch"]
PRODUCTS=["Inuka 4 weeks","Fadhili 6 weeks","Kuza 12 weeks","AgriAdvance 16 weeks","FlexiLoan 8 weeks","BizBoost 10 weeks"]

def generate_id(prefix=None):
    return str(prefix)+str(random.randint(0,999999))[:8] if prefix else str(random.randint(1000000,99999999))

def generate_phone(): return "+2547"+str(random.randint(1000000,9999999))

def make_dataset(n=2000, default_rate=0.25, seed=42, include_history=True):
    random.seed(seed); rows=[]
    for i in range(n):
        name=faker.first_name()+" "+faker.last_name()
        nid=generate_id(random.choice([32,33,34,35])) if random.random()<0.5 else generate_id()
        branch,prod=random.choice(BRANCHES),random.choice(PRODUCTS)
        income=random.choice([15000,20000,30000,50000,80000,120000,200000])
        amount=random.randint(2000,600000)
        created=datetime.utcnow()-timedelta(days=random.randint(0,730))
        status="default" if random.random()<default_rate else "performing"
        rows.append(dict(client_id=i,name=name,national_id=nid,phone=generate_phone(),town=random.choice(KENYAN_TOWNS),
                         branch=branch,product=prod,income=income,loan_amount=amount,loan_status=status,
                         created_date=created.strftime("%Y-%m-%d")))
    return pd.DataFrame(rows)
"""
write_module(SYNTH_DIR/"generator.py", synth_py)

# -------- 4) ml/engine.py --------
ml_engine_py = r"""
import os,json,joblib,hashlib,time
import numpy as np
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

MODELS_DIR="/content/loan_app/models"
def _fingerprint(cols): return hashlib.sha1("|".join(sorted(cols)).encode()).hexdigest()[:12]

class HybridModel:
    def __init__(self,models): self.models=models
    def predict_proba(self,X):
        arr=X.values if hasattr(X,"values") else np.asarray(X)
        probs=[]
        for m in self.models:
            try: p=m.predict_proba(arr)[:,1]
            except:
                try: p=1/(1+np.exp(-m.decision_function(arr)))
                except: p=np.zeros(arr.shape[0])
            probs.append(p)
        avg=np.mean(np.vstack(probs),axis=0)
        return np.vstack([1-avg,avg]).T
    def predict(self,X): return (self.predict_proba(X)[:,1]>0.5).astype(int)

def train_baseline(X,y,name="baseline",random_state=42):
    Xt,Xv,yt,yv=train_test_split(X,y,test_size=0.25,random_state=random_state,stratify=y if len(set(y))>1 else None)
    try: Xb,yb=SMOTE(random_state=random_state).fit_resample(Xt,yt)
    except: Xb,yb=Xt,yt
    lr=LogisticRegression(max_iter=1000)
    sgd=CalibratedClassifierCV(SGDClassifier(max_iter=2000),cv=3)
    xgb=XGBClassifier(use_label_encoder=False,eval_metric="logloss",verbosity=0)
    for m in [lr,sgd,xgb]:
        try: m.fit(Xb,yb)
        except: pass
    hybrid=HybridModel([lr,sgd,xgb])
    try: auc=float(roc_auc_score(yv,hybrid.predict_proba(Xv)[:,1]))
    except: auc=0.0
    meta={"name":name,"created_at":time.time(),"features":list(X.columns),"fingerprint":_fingerprint(X.columns),"auc":auc}
    os.makedirs(MODELS_DIR,exist_ok=True)
    joblib.dump(hybrid,f"{MODELS_DIR}/{name}.pkl")
    json.dump(meta,open(f"{MODELS_DIR}/{name}.meta.json","w"))
    return {"auc":auc,"model_path":f"{MODELS_DIR}/{name}.pkl"}
"""
write_module(ML_DIR/"engine.py", ml_engine_py)

# -------- 5) ml/utils.py --------
ml_utils_py = r"""
import shap, numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score,precision_recall_curve,auc
def evaluate(model,X,y):
    try:
        p=model.predict_proba(X)[:,1]
        prec,rec,_=precision_recall_curve(y,p)
        return {"auc":float(roc_auc_score(y,p)),"acc":float(accuracy_score(y,(p>0.5).astype(int))),
                "pr_auc":float(auc(rec,prec))}
    except Exception as e: return {"error":str(e)}
def explain(model,X,n=200):
    try:
        Xs=X.sample(min(n,len(X)))
        base=model.models[2] if hasattr(model,"models") else model
        return shap.Explainer(base,Xs)(Xs)
    except Exception as e: return {"error":str(e)}
"""
write_module(ML_DIR/"utils.py", ml_utils_py)

# -------- 6) ml/audit.py --------
ml_audit_py = r"""
import sqlite3,os,datetime
DB_PATH="/content/loan_app/data/audit.db"
def init_audit():
    os.makedirs(os.path.dirname(DB_PATH),exist_ok=True)
    conn=sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute('CREATE TABLE IF NOT EXISTS audit_log (ts TEXT,user TEXT,action TEXT,details TEXT)')
    conn.commit(); conn.close()
def log(user,action,details=""):
    conn=sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute('INSERT INTO audit_log VALUES (?,?,?,?)',(datetime.datetime.utcnow().isoformat(),user,action,details))
    conn.commit(); conn.close()
"""
write_module(ML_DIR/"audit.py", ml_audit_py)

# -------- init files --------
for p in [MODULES,SYNTH_DIR,ML_DIR,STREAMLIT_DIR]: (p/"__init__.py").write_text("# init\n")

# -------- Print repo tree --------
def print_tree(root=BASE,depth=3):
    for p,_,files in os.walk(root):
        level=p.replace(str(root),"").count(os.sep)
        if level<=depth:
            print("  "*level+os.path.basename(p)+"/")
            for f in files: print("  "*level+"  - "+f)
print_tree()

# -------- Import checks --------
for mod in ["modules.auth","modules.schema","modules.synth.generator","modules.ml.engine","modules.ml.utils","modules.ml.audit"]:
    try: importlib.import_module(mod); print("✅",mod,"OK")
    except Exception as e: print("❌",mod,"->",e)

loan_app/
  data/
  logs/
  modules/
    - auth.py
    - __init__.py
    - schema.py
    ml/
      - audit.py
      - utils.py
      - __init__.py
      - engine.py
    streamlit_app/
      - __init__.py
    synth/
      - __init__.py
      - generator.py
  models/
✅ modules.auth OK
❌ modules.schema -> numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
❌ modules.synth.generator -> numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
❌ modules.ml.engine -> No module named 'numpy.char'
❌ modules.ml.utils -> numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
✅ modules.ml.audit OK


In [ ]:
# -------- Cell 3/9: Write Streamlit App Skeleton --------
# Paste into Colab and run. This writes app.py into the repo and does a simple import-check.

import os, textwrap
BASE = "/content/loan_app"
APP_DIR = os.path.join(BASE, "modules", "streamlit_app")
os.makedirs(APP_DIR, exist_ok=True)
APP_PATH = os.path.join(APP_DIR, "app.py")

app_src = r'''
"""
Streamlit app skeleton for Loan IQ
- Tabs: Login/Register | Upload & Ingest | Client Dashboard | Admin Sandbox | Global Insights | Reports
- Defensive imports: if a module is missing, shows guidance in UI
- Admin credentials preserved: Admin / Shady868
"""

import streamlit as st
import os, io, json, time
from pathlib import Path

BASE = Path("/content/loan_app")
DATA_DIR = BASE / "data"
UPLOAD_DIR = DATA_DIR / "uploads"
MODELS_DIR = BASE / "models"

UPLOAD_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# -- defensive imports --
missing = []
try:
    from modules import auth
except Exception as e:
    auth = None
    missing.append(("auth", str(e)))
try:
    from modules import schema
except Exception as e:
    schema = None
    missing.append(("schema", str(e)))
try:
    from modules.synth import generator as synth_generator
except Exception as e:
    synth_generator = None
    missing.append(("synth.generator", str(e)))
try:
    from modules.ml import engine as ml_engine
except Exception as e:
    ml_engine = None
    missing.append(("ml.engine", str(e)))
try:
    from modules.ml import utils as ml_utils
except Exception as e:
    ml_utils = None
    missing.append(("ml.utils", str(e)))
try:
    from modules.ml import audit as ml_audit
except Exception as e:
    ml_audit = None
    missing.append(("ml.audit", str(e)))

# initialize DBs if available
if auth:
    try:
        auth.init_db()
    except Exception:
        pass
if ml_audit:
    try:
        ml_audit.init_audit_db()
    except Exception:
        pass

# ----------------- Helpers -----------------
def show_missing_modules():
    st.error("Some backend modules failed to import. The app will be degraded. See details below.")
    for name, err in missing:
        st.text(f"{name}: {err}")

def save_upload(file_bytes, filename):
    p = UPLOAD_DIR / filename
    with open(p, "wb") as f:
        f.write(file_bytes)
    return str(p)

def load_uploaded_dataframe(path):
    try:
        import pandas as pd
        if str(path).lower().endswith((".xls", ".xlsx")):
            return pd.read_excel(path)
        return pd.read_csv(path)
    except Exception as e:
        st.error(f"Failed to load uploaded file: {e}")
        return None

def ensure_models_folder():
    MODELS_DIR.mkdir(parents=True, exist_ok=True)

# ----------------- Auth helpers -----------------
def login_flow():
    st.subheader("Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    if st.button("Login"):
        if auth:
            ok, role = auth.authenticate(username, password)
            if ok:
                st.session_state['user'] = username
                st.session_state['role'] = role
                st.success(f"Logged in as {username} ({role})")
                if ml_audit:
                    ml_audit.log(username, "login", "web login")
            else:
                st.error("Invalid credentials")
        else:
            st.error("Auth module missing")

def register_flow():
    st.subheader("Register (company)")
    r_user = st.text_input("New username", key="r_user")
    r_pass = st.text_input("New password", type="password", key="r_pass")
    if st.button("Register"):
        if auth:
            auth.register_user(r_user, r_pass, role="company")
            st.success("User registered (company). Admin must approve if needed.")
        else:
            st.error("Auth module missing")

# ----------------- Upload & Ingest -----------------
def upload_and_preview():
    st.header("Upload dataset (CSV/XLSX)")
    uploaded = st.file_uploader("Choose a file", type=["csv","xlsx"])
    if uploaded is not None:
        bytes_data = uploaded.getvalue()
        filename = uploaded.name
        saved = save_upload(bytes_data, filename)
        st.success(f"Saved to {saved}")
        df = load_uploaded_dataframe(saved)
        if df is not None:
            st.write("Preview:")
            st.dataframe(df.head(10))
            # attempt schema.prepare_for_ml if available
            if schema:
                try:
                    X, y, enriched = schema.prepare_for_ml(df, aggregate_by_client=False)
                    st.write("Enriched preview (first 10 rows):")
                    st.dataframe(enriched.head(10))
                    st.write("Feature matrix shape:", X.shape)
                except Exception as e:
                    st.warning(f"prepare_for_ml failed: {e}")
            else:
                st.info("Schema module not available; raw preview shown.")

# ----------------- Client Dashboard -----------------
def client_dashboard():
    st.header("Client Dashboard")
    st.info("Lookup by national ID (unique identifier). This view supports multiple loans per ID.")
    nid = st.text_input("Enter National ID")
    if st.button("Lookup"):
        # naive search across uploads
        found = []
        for p in UPLOAD_DIR.glob("*"):
            try:
                import pandas as pd
                if str(p).lower().endswith((".xls",".xlsx")):
                    df = pd.read_excel(p)
                else:
                    df = pd.read_csv(p)
                # standardize col names
                if schema:
                    df2 = schema.coerce_types_and_derive(df)
                else:
                    df2 = df
                if "national_id" in df2.columns:
                    matches = df2[df2["national_id"].astype(str).str.contains(str(nid))]
                    if not matches.empty:
                        found.append((p.name, matches))
            except Exception as e:
                st.write(f"Error reading {p}: {e}")
        if not found:
            st.warning("No loans found for that ID in uploaded files.")
        else:
            for fname, dfm in found:
                st.subheader(f"Matches in {fname}")
                st.dataframe(dfm.head(50))
                # show aggregates
                try:
                    agg = dfm.groupby("national_id").agg({
                        "loan_amount_num":"sum",
                        "is_default":"max",
                        "days_since_issue":"min"
                    })
                    st.write("Aggregates:")
                    st.dataframe(agg)
                except Exception as e:
                    st.write(f"Aggregation failed: {e}")

# ----------------- Admin Sandbox -----------------
def admin_sandbox():
    st.header("Admin Sandbox (admin only)")
    user = st.session_state.get("user")
    role = st.session_state.get("role")
    if user != "Admin":
        st.warning("Admin sandbox is restricted to Admin user. Please login as Admin to continue.")
        return

    st.subheader("Impersonation")
    imp = st.text_input("Impersonate username (type in username to impersonate)")
    if st.button("Impersonate"):
        if auth:
            st.session_state["impersonate"] = imp
            st.success(f"Now impersonating {imp}")
            if ml_audit: ml_audit.log("Admin", "impersonate", imp)
        else:
            st.error("Auth module missing")

    st.subheader("Synthetic Data")
    n = st.number_input("Rows to generate", min_value=100, max_value=20000, value=2000, step=100)
    default_rate = st.slider("Default rate", 0.0, 1.0, 0.25)
    if st.button("Generate synthetic dataset"):
        if synth_generator:
            df = synth_generator.make_dataset(int(n), default_rate=default_rate, include_history=True)
            p = UPLOAD_DIR / f"synthetic_{int(time.time())}.csv"
            df.to_csv(p, index=False)
            st.success(f"Generated synthetic dataset: {p}")
            st.dataframe(df.head())
            if ml_audit: ml_audit.log("Admin", "synth_generate", f"{p}")
        else:
            st.error("Synth generator missing")

    st.subheader("Model Training / Registry")
    model_name = st.text_input("Model name", value="baseline")
    if st.button("Train baseline model"):
        # find most recent upload to use for training
        files = sorted(UPLOAD_DIR.glob("*"), key=lambda p: p.stat().st_mtime, reverse=True)
        if not files:
            st.error("No uploaded datasets found for training")
        else:
            # load first file
            import pandas as pd
            p = files[0]
            try:
                if str(p).lower().endswith((".xls",".xlsx")):
                    df = pd.read_excel(p)
                else:
                    df = pd.read_csv(p)
                st.info(f"Using {p.name} for training")
                if schema:
                    X, y, enriched = schema.prepare_for_ml(df, aggregate_by_client=False)
                else:
                    st.error("Schema module required for training")
                    return
                st.write("Feature sample:")
                st.dataframe(X.head())
                ensure_models_folder()
                if ml_engine:
                    res = ml_engine.train_baseline(X, y, name=model_name)
                    st.success(f"Training complete. AUC: {res.get('auc'):.4f}")
                    st.write("Model artifact:", res.get("model_path"))
                    if ml_audit: ml_audit.log(st.session_state.get("user","unknown"), "train_model", json.dumps(res))
                else:
                    st.error("ML engine missing")
            except Exception as e:
                st.error(f"Training pipeline failed: {e}")

    st.subheader("Model Registry / Pin / Impersonate")
    # list models
    models = [p for p in MODELS_DIR.glob("*.meta.json")]
    for m in models:
        try:
            meta = json.load(open(m))
            st.write(meta)
            if st.button(f"Pin {meta.get('name')}"):
                # write production pointer
                prod = MODELS_DIR / "PROD_MODEL.txt"
                prod.write_text(meta.get("name"))
                st.success(f"Pinned {meta.get('name')} as production")
                if ml_audit: ml_audit.log("Admin", "pin_model", meta.get("name"))
        except Exception as e:
            st.write(f"Failed to read model meta {m}: {e}")

# ----------------- Global Insights -----------------
def global_insights():
    st.header("Global Insights")
    st.info("KPI snapshots across uploaded datasets")
    # simple aggregate across all uploads
    import pandas as pd
    dfs = []
    for p in UPLOAD_DIR.glob("*"):
        try:
            if str(p).lower().endswith((".xls",".xlsx")):
                df = pd.read_excel(p)
            else:
                df = pd.read_csv(p)
            if schema:
                df2 = schema.coerce_types_and_derive(df)
                dfs.append(df2)
        except Exception as e:
            st.write(f"Failed reading {p}: {e}")
    if not dfs:
        st.warning("No enriched uploads available")
        return
    full = pd.concat(dfs, ignore_index=True)
    st.write("Combined data sample:")
    st.dataframe(full.head())
    # KPIs
    total_loans = len(full)
    total_default = int(full["is_default"].sum()) if "is_default" in full.columns else 0
    avg_loan = float(full["loan_amount_num"].mean()) if "loan_amount_num" in full.columns else 0.0
    st.metric("Total loans", total_loans)
    st.metric("Total defaults", total_default)
    st.metric("Avg loan amount", f"{avg_loan:,.0f}")

# ----------------- Reports -----------------
def reports_tab():
    st.header("Reports & Exports")
    st.write("Export aggregated CSVs, per-client reports, or model predictions (CSV).")
    if st.button("Export client-level aggregates (CSV)"):
        import pandas as pd
        dfs = []
        for p in UPLOAD_DIR.glob("*"):
            try:
                if str(p).lower().endswith((".xls",".xlsx")):
                    df = pd.read_excel(p)
                else:
                    df = pd.read_csv(p)
                if schema:
                    X, y, enriched = schema.prepare_for_ml(df, aggregate_by_client=True)
                    dfs.append(pd.concat([X.reset_index(), y.reset_index(drop=False)], axis=1))
            except Exception as e:
                st.write(f"Skipping {p}: {e}")
        if not dfs:
            st.warning("No data to export")
            return
        out = pd.concat(dfs, ignore_index=True)
        out_path = UPLOAD_DIR / f"client_aggregates_{int(time.time())}.csv"
        out.to_csv(out_path, index=False)
        st.success(f"Exported: {out_path}")

# ----------------- Main layout -----------------
def main():
    st.set_page_config(page_title="Loan IQ", layout="wide")
    st.title("Loan IQ — Loan company portal + Admin Sandbox")

    if missing:
        show_missing_modules()
        st.stop()

    # session init
    if "user" not in st.session_state:
        st.session_state["user"] = None
        st.session_state["role"] = None
    # top-level nav
    menu = ["Home / Login", "Upload & Ingest", "Client Dashboard", "Admin Sandbox", "Global Insights", "Reports"]
    choice = st.sidebar.selectbox("Menu", menu)

    if choice == "Home / Login":
        col1, col2 = st.columns(2)
        with col1:
            login_flow()
        with col2:
            register_flow()
        # quick links
        st.write("Quick actions:")
        if st.button("Generate small synthetic (100 rows)"):
            if synth_generator:
                df = synth_generator.make_dataset(100, default_rate=0.2)
                p = UPLOAD_DIR / f"synthetic_small_{int(time.time())}.csv"
                df.to_csv(p, index=False)
                st.success(f"Saved small synthetic to {p}")
            else:
                st.error("Synth generator missing")

    elif choice == "Upload & Ingest":
        upload_and_preview()

    elif choice == "Client Dashboard":
        client_dashboard()

    elif choice == "Admin Sandbox":
        admin_sandbox()

    elif choice == "Global Insights":
        global_insights()

    elif choice == "Reports":
        reports_tab()

if __name__ == "__main__":
    main()
'''

with open(APP_PATH, "w") as f:
    f.write(app_src)

print("Wrote Streamlit app to:", APP_PATH)
# quick import-check (non-executing)
print("Attempting lightweight import checks for modules used by the app (no heavy work).")
errs = []
try:
    import importlib
    importlib.invalidate_caches()
    for mod in ["modules.auth", "modules.schema", "modules.synth.generator", "modules.ml.engine", "modules.ml.utils", "modules.ml.audit"]:
        try:
            importlib.import_module(mod)
            print("Imported:", mod)
        except Exception as e:
            print("Import failed:", mod, "->", e)
            errs.append((mod, str(e)))
except Exception as e:
    print("Import-check harness error:", e)

if errs:
    print("\\nSome imports failed (they will show in the app). You can inspect the file at", APP_PATH)
else:
    print("\\nAll light imports succeeded. App skeleton ready. Next: Cell 4 will add CLI helpers, ngrok helpers, and a small runner script.")

Wrote Streamlit app to: /content/loan_app/modules/streamlit_app/app.py
Attempting lightweight import checks for modules used by the app (no heavy work).
Imported: modules.auth
Import failed: modules.schema -> numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
Import failed: modules.synth.generator -> numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
Import failed: modules.ml.engine -> No module named 'numpy.char'
Import failed: modules.ml.utils -> numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject
Imported: modules.ml.audit
\nSome imports failed (they will show in the app). You can inspect the file at /content/loan_app/modules/streamlit_app/app.py


In [ ]:
# === One-Time Environment & App Patcher ===
import os, sys, subprocess, importlib, shutil

print("🔧 Resetting environment...")

# --- 1. Fix Python paths & create base folders ---
os.makedirs("modules/synth", exist_ok=True)
os.makedirs("modules/ml", exist_ok=True)
os.makedirs("modules/streamlit_app/pages", exist_ok=True)
os.makedirs("data/sandbox_batches", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("logs", exist_ok=True)

if "modules" not in sys.path:
    sys.path.append("modules")

# --- 2. Install deps (skip if installed) ---
def pip_install(package, version=None):
    try:
        if version:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package}=={version}", "--quiet"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
    except Exception as e:
        print(f"⚠️ Could not install {package}: {e}")

print("📦 Ensuring dependencies are installed...")
deps = {
    "numpy": "1.26.4",   # stable for sklearn/xgboost in Colab
    "pandas": None,
    "scikit-learn": None,
    "xgboost": None,
    "shap": None,
    "imbalanced-learn": None,
    "faker": None,
    "streamlit": None,
    "pyngrok": None,
}
for pkg, ver in deps.items():
    pip_install(pkg, ver)

# --- 3. Admin & Ngrok ---
ADMIN_USER = "Admin"
ADMIN_PASS = "Shady868"
NGROK_AUTHTOKEN = "31rYvgklL0EdX9bGLvTXc313efE_2GyDFGPUNAyFgB83bikTF"

# Save them for app use
with open("modules/streamlit_app/config.py", "w") as f:
    f.write(f"ADMIN_USER = '{ADMIN_USER}'\n")
    f.write(f"ADMIN_PASS = '{ADMIN_PASS}'\n")
    f.write(f"NGROK_AUTHTOKEN = '{NGROK_AUTHTOKEN}'\n")

# --- 4. SQLite DB reset ---
import sqlite3
conn = sqlite3.connect("data/app.db")
cur = conn.cursor()

# Drop old tables (safe reset)
for t in ["users", "audit_logs"]:
    cur.execute(f"DROP TABLE IF EXISTS {t}")

cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password TEXT,
    role TEXT DEFAULT 'client'
)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS audit_logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    action TEXT,
    user TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")
# Ensure admin is present
import hashlib
hashed_pw = hashlib.sha256(ADMIN_PASS.encode()).hexdigest()
cur.execute("INSERT OR IGNORE INTO users (username, password, role) VALUES (?, ?, ?)",
            (ADMIN_USER, hashed_pw, "admin"))
conn.commit()
conn.close()
print("🗄️ Database ready with admin user.")

# --- 5. Kenyan Faker setup ---
from faker import Faker
from faker.providers import BaseProvider

faker = Faker("en_US")  # fallback locale

class KenyanProvider(BaseProvider):
    kenyan_first_names_male = [
        "Otieno", "Kamau", "Ochieng", "Mwangi", "Odhiambo", "Mutiso"
    ]
    kenyan_first_names_female = [
        "Achieng", "Wanjiku", "Nyambura", "Atieno", "Naliaka", "Wairimu"
    ]
    kenyan_last_names = [
        "Omondi", "Njoroge", "Mutua", "Omondi", "Chebet", "Barasa"
    ]
    kenyan_branches = [
        "Nairobi", "Mombasa", "Kisumu", "Nakuru", "Eldoret", "Thika"
    ]

    def kenyan_name(self):
        if self.random_element([True, False]):
            return f"{self.random_element(self.kenyan_first_names_male)} {self.random_element(self.kenyan_last_names)}"
        else:
            return f"{self.random_element(self.kenyan_first_names_female)} {self.random_element(self.kenyan_last_names)}"

    def kenyan_branch(self):
        return self.random_element(self.kenyan_branches)

    def gov_id(self):
        return str(self.random_int(min=10000000, max=99999999))

faker.add_provider(KenyanProvider)

# Test samples
print("🌍 Faker localized to Kenya with custom provider:")
print("Name:", faker.kenyan_name())
print("Branch:", faker.kenyan_branch())
print("Gov ID:", faker.gov_id())
# --- 7. Verify imports ---
import modules.synth, modules.ml
print("✅ Modules folder intact and importable.")

print("\n🚀 PATCH COMPLETE. You're ready to continue building.")
print("👉 Next: you can safely paste large code blocks (700-900 lines) without worrying about missing paths/errors.")

🔧 Resetting environment...
📦 Ensuring dependencies are installed...
🗄️ Database ready with admin user.
🌍 Faker localized to Kenya with custom provider:
Name: Achieng Barasa
Branch: Kisumu
Gov ID: 98271355
✅ Modules folder intact and importable.

🚀 PATCH COMPLETE. You're ready to continue building.
👉 Next: you can safely paste large code blocks (700-900 lines) without worrying about missing paths/errors.


In [ ]:
# -------- Cell 4/9: Runner & Ngrok --------
import os, sys
from pathlib import Path

BASE = Path("/content/loan_app")
RUNNER = BASE / "run_app.py"
APP_FILE = BASE / "modules" / "streamlit_app" / "app.py"

runner_code = r"""
import os, sys, time
from pyngrok import ngrok

APP_PATH = "/content/loan_app/modules/streamlit_app/app.py"

def main():
    if not os.path.exists(APP_PATH):
        print("❌ App not found:", APP_PATH); sys.exit(1)

    ngrok.kill()  # kill old tunnels

    public_url=None
    for attempt in range(5):
        try:
            tunnel=ngrok.connect(8501,"http"); public_url=tunnel.public_url; break
        except Exception as e:
            print(f"ngrok attempt {attempt+1}/5 failed:",e); time.sleep(3)
    if not public_url:
        print("❌ ngrok failed after retries"); sys.exit(1)

    print(f"✅ App will be live at: {public_url}\\n")

    os.system(f"streamlit run {APP_PATH} --server.port 8501 --server.headless true")

if __name__=="__main__": main()
"""
RUNNER.write_text(runner_code.strip()+"\n")

# Quick smoke check
if APP_FILE.exists():
    print("✅ Runner written:", RUNNER)
    print("👉 Later, launch with: !python /content/loan_app/run_app.py")
else:
    print("❌ Streamlit app missing, re-run Cell 3 first")

✅ Runner written: /content/loan_app/run_app.py
👉 Later, launch with: !python /content/loan_app/run_app.py


In [ ]:
# -------- Cell 5/9: Utility Scripts & Smoke Tests --------
import os, sys
from pathlib import Path

BASE = Path("/content/loan_app")
UTILS_DIR = BASE / "utils"
UTILS_DIR.mkdir(exist_ok=True)

# --- inspect_data.py ---
inspect_code = r"""
import sys, pandas as pd
from modules import schema
from pathlib import Path

UPLOAD_DIR = Path("/content/loan_app/data/uploads")

def inspect_latest():
    files=sorted(UPLOAD_DIR.glob("*"), key=lambda p:p.stat().st_mtime, reverse=True)
    if not files:
        print("❌ No uploaded files to inspect."); return
    f=files[0]; print("Inspecting:",f)
    df=pd.read_csv(f) if f.suffix==".csv" else pd.read_excel(f)
    print("Raw shape:",df.shape)
    X,y,en=schema.prepare_for_ml(df)
    print("Enriched shape:",en.shape," Features:",X.shape," Target balance:",y.value_counts().to_dict())

if __name__=="__main__": inspect_latest()
"""
(BASE/"utils"/"inspect_data.py").write_text(inspect_code.strip()+"\n")

# --- train_smoke.py ---
train_code = r"""
import sys, pandas as pd
from modules.synth import generator
from modules import schema
from modules.ml import engine

def smoke_train():
    print("Generating synthetic dataset (500 rows)...")
    df=generator.make_dataset(500, default_rate=0.3)
    X,y,en=schema.prepare_for_ml(df)
    print("Training baseline model...")
    res=engine.train_baseline(X,y,name="smoke_test")
    print("✅ Smoke train done. AUC:",res.get("auc"))

if __name__=="__main__": smoke_train()
"""
(BASE/"utils"/"train_smoke.py").write_text(train_code.strip()+"\n")

# --- Smoke tests ---
print("Running smoke test: inspect_data.py (no files yet expected)...")
os.system("python /content/loan_app/utils/inspect_data.py || true")

print("\nRunning smoke test: train_smoke.py...")
os.system("python /content/loan_app/utils/train_smoke.py || true")

print("\n✅ Utilities ready. You can run them manually from Colab shell.")

Running smoke test: inspect_data.py (no files yet expected)...

Running smoke test: train_smoke.py...

✅ Utilities ready. You can run them manually from Colab shell.


In [ ]:
# -------- Cell 6/9: Enhanced Kenyan Synthetic Generator + Visuals (50 towns + regions) --------
import os, sys, random
from pathlib import Path
BASE = Path("/content/loan_app")
SYNTH_DIR = BASE / "modules" / "synth"
VIS_DIR = BASE / "modules" / "visuals"
LOG_PLOTS = BASE / "logs" / "plots"
LOG_PLOTS.mkdir(parents=True, exist_ok=True)
VIS_DIR.mkdir(parents=True, exist_ok=True)

def write_mod(path: Path, code: str):
    path.write_text(code.strip() + "\n")
    with open(path) as f: f.read()

# ---------- enhanced generator ----------
enhanced_gen = r"""
import random
from faker import Faker
from datetime import datetime, timedelta
import pandas as pd

faker = Faker()
Faker.seed = lambda s: random.seed(s)

# ~50 towns mapped to regions
TOWN_REGION = {
    "Nairobi":"Nairobi","Thika":"Central","Nyeri":"Central","Murang'a":"Central","Kiambu":"Central",
    "Machakos":"Eastern","Embu":"Eastern","Meru":"Eastern","Kitui":"Eastern","Mwingi":"Eastern",
    "Nakuru":"Rift Valley","Naivasha":"Rift Valley","Kericho":"Rift Valley","Eldoret":"Rift Valley","Bomet":"Rift Valley","Narok":"Rift Valley","Kajiado":"Rift Valley",
    "Kisumu":"Nyanza","Kisii":"Nyanza","Homabay":"Nyanza","Migori":"Nyanza","Siaya":"Nyanza",
    "Mombasa":"Coast","Kilifi":"Coast","Malindi":"Coast","Kwale":"Coast","Lamu":"Coast","Voi":"Coast",
    "Garissa":"North Eastern","Wajir":"North Eastern","Mandera":"North Eastern",
    "Kakamega":"Western","Bungoma":"Western","Busia":"Western","Vihiga":"Western","Trans Nzoia":"Western",
    "Turkana":"Rift Valley","West Pokot":"Rift Valley","Isiolo":"Eastern","Marsabit":"Eastern","Samburu":"Rift Valley",
    "Taita Taveta":"Coast","Taveta":"Coast","Loitoktok":"Rift Valley","Gilgil":"Rift Valley","Kerugoya":"Central"
}

# Weighted regions: Nairobi, Eastern, Rift, Central get more weight
REGION_WEIGHTS = {
    "Nairobi":0.20,"Eastern":0.20,"Rift Valley":0.20,"Central":0.20,
    "Nyanza":0.07,"Western":0.06,"Coast":0.05,"North Eastern":0.02
}

def _pick_town():
    regions = list(REGION_WEIGHTS.keys())
    region = random.choices(regions, weights=[REGION_WEIGHTS[r] for r in regions])[0]
    towns = [t for t,r in TOWN_REGION.items() if r==region]
    town = random.choice(towns)
    return town, region

def _skewed_loan_amount(min_k=5000, max_k=70000, skew=1.8):
    r = random.random() ** skew
    return int(min_k + r * (max_k - min_k))

def generate_national_id(prefix_choices=None):
    if prefix_choices is None: prefix_choices=[32,33,34,35,36]
    prefix = str(random.choice(prefix_choices))
    tail = str(random.randint(0,999999)).zfill(6)
    return prefix + tail

def make_dataset(n=2000, default_rate=0.20, multi_loan_frac=0.30, female_frac=0.60,
                 loan_min=5000, loan_max=70000, seed=42):
    random.seed(seed); rows=[]
    for i in range(n):
        gender = "female" if random.random() < female_frac else "male"
        first = faker.first_name_female() if gender=="female" else faker.first_name_male()
        last = faker.last_name()
        name = f"{first} {last}"
        nid = generate_national_id()
        phone = "+2547" + str(random.randint(10000000,99999999))[1:9]
        town, region = _pick_town()
        product = random.choice(["Inuka 4 weeks","Fadhili 6 weeks","Kuza 12 weeks","AgriAdvance 16 weeks","FlexiLoan 8 weeks","BizBoost 10 weeks","QuickPay 2 weeks"])
        income = random.choice([12000,15000,20000,25000,30000,40000,60000])
        amount = _skewed_loan_amount(loan_min, loan_max, skew=1.8)
        created = datetime.utcnow() - timedelta(days=random.randint(0,730))
        dobias = default_rate + (0.02 if amount > 40000 else -0.01)
        status = "default" if random.random() < max(0, min(0.95, dobias)) else "performing"
        rows.append({
            "client_id": i, "name": name, "national_id": nid, "phone": phone,
            "town": town, "region": region, "product": product,
            "income": income, "loan_amount": amount, "loan_status": status,
            "created_date": created.strftime("%Y-%m-%d"), "gender": gender
        })
        if random.random() < multi_loan_frac:
            extra_amount = int(amount * random.uniform(0.3, 1.2))
            created2 = created - timedelta(days=random.randint(30, 900))
            rows.append({
                "client_id": i, "name": name, "national_id": nid, "phone": phone,
                "town": town, "region": region, "product": random.choice(["Inuka 4 weeks","Fadhili 6 weeks","Kuza 12 weeks","AgriAdvance 16 weeks","FlexiLoan 8 weeks","BizBoost 10 weeks","QuickPay 2 weeks"]),
                "income": income, "loan_amount": extra_amount, "loan_status": "performing",
                "created_date": created2.strftime("%Y-%m-%d"), "gender": gender
            })
    return pd.DataFrame(rows)
"""
write_mod(SYNTH_DIR / "enhanced_generator.py", enhanced_gen)

# ---------- visuals (same as before, package) ----------
visuals_py = r"""
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

PLOT_DIR = Path('/content/loan_app/logs/plots')
PLOT_DIR.mkdir(parents=True, exist_ok=True)

def kpis_from_df(df):
    total_loans = len(df)
    defaults = int(df.get('loan_status', pd.Series()).astype(str).str.lower().str.contains('default').sum()) if 'loan_status' in df else int(df.get('is_default', pd.Series()).sum() if 'is_default' in df else 0)
    avg_loan = float(df.get('loan_amount', pd.Series()).astype(float).mean()) if 'loan_amount' in df else 0.0
    female_share = float((df.get('gender','').astype(str)=='female').mean()) if 'gender' in df else np.nan
    multi_loans = df.groupby(df.get('national_id', df.get('client_id'))).size().gt(1).mean()
    return {'total_loans':total_loans,'defaults':defaults,'avg_loan':avg_loan,'female_share':female_share,'multi_loan_frac':float(multi_loans)}

def plot_loan_size_hist(df, filename='loan_size_hist.png'):
    vals = df['loan_amount'].astype(float)
    fig, ax = plt.subplots()
    ax.hist(vals, bins=30)
    ax.set_title('Loan size distribution'); ax.set_xlabel('Loan amount'); ax.set_ylabel('Count')
    out = PLOT_DIR / filename
    fig.savefig(out, bbox_inches='tight'); plt.close(fig); return str(out)

def plot_defaults_pie(df, filename='defaults_pie.png'):
    status = df['loan_status'].astype(str).str.lower().apply(lambda s: 'default' if 'default' in s else 'performing')
    counts = status.value_counts()
    fig, ax = plt.subplots()
    ax.pie(counts.values, labels=counts.index, autopct='%1.1f%%')
    ax.set_title('Default vs Performing')
    out = PLOT_DIR / filename
    fig.savefig(out, bbox_inches='tight'); plt.close(fig); return str(out)

def plot_time_series_by_month(df, filename='loans_by_month.png'):
    if 'created_date' not in df.columns and 'created_date_parsed' in df.columns:
        df['created_date'] = df['created_date_parsed']
    df['created_date_parsed'] = pd.to_datetime(df['created_date'], errors='coerce')
    df['month'] = df['created_date_parsed'].dt.to_period('M')
    counts = df.groupby('month').size()
    fig, ax = plt.subplots()
    ax.plot(counts.index.to_timestamp(), counts.values)
    ax.set_title('Loans by month'); ax.set_ylabel('Count')
    out = PLOT_DIR / filename
    fig.savefig(out, bbox_inches='tight'); plt.close(fig); return str(out)

def safe_shap_plot(shap_values, feature_names=None, out_name='shap_summary.png'):
    try:
        import shap
        fig = shap.plots.bar(shap_values, show=False)
        out = PLOT_DIR / out_name
        fig.figure.savefig(out, bbox_inches='tight')
        return str(out)
    except Exception as e:
        p = PLOT_DIR / out_name
        with open(p,'w') as f: f.write('shap failed: '+str(e))
        return str(p)
"""
(VIS_DIR / "__init__.py").write_text(visuals_py.strip()+"\n")

# ---------- Smoke test ----------
print("Running smoke test for enhanced generator + visuals (50 towns + regions)...")
try:
    import importlib
    importlib.invalidate_caches()
    ig = importlib.import_module("modules.synth.enhanced_generator")
    import modules.visuals as vis
    importlib.reload(vis)  # force reload so new functions appear
    schema = importlib.import_module("modules.schema")

    df = ig.make_dataset(n=1000, default_rate=0.20, multi_loan_frac=0.30,
                         female_frac=0.60, seed=123)
    csvp = (BASE/"data/uploads/enhanced_synth_regions.csv")
    df.to_csv(csvp, index=False)
    print("Generated dataset saved to:", csvp)

    X,y,en = schema.prepare_for_ml(df)
    print("Enriched shape:", en.shape, "Feature matrix:", X.shape)

    kpis = vis.kpis_from_df(df)
    print("KPIs:", kpis)

    p1 = vis.plot_loan_size_hist(df)
    p2 = vis.plot_defaults_pie(df)
    p3 = vis.plot_time_series_by_month(df)
    print("Plots written:", p1, p2, p3)

    print("Cell 6 smoke test: ✅ OK")
except Exception as e:
    print("Cell 6 smoke test failed:", e)
    raise

Running smoke test for enhanced generator + visuals (50 towns + regions)...
Cell 6 smoke test failed: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# -------- Cell 7/9: Client Reports + SHAP Integration (clean + fallback) --------
import os, json, joblib, sys, warnings
from pathlib import Path

BASE = Path("/content/loan_app")
REPORTS_DIR = BASE / "modules" / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

reports_py = r"""
import os, pandas as pd, json, joblib, numpy as np, warnings
from pathlib import Path
from modules import schema
from modules.ml import engine, utils
from modules.visuals import kpis_from_df

OUT_DIR = Path('/content/loan_app/logs/reports')
OUT_DIR.mkdir(parents=True, exist_ok=True)
warnings.filterwarnings("ignore")  # suppress sklearn/xgboost spam

def _fallback_features(model, X):
    try:
        if hasattr(model, "coef_"):
            coefs = model.coef_[0] if len(model.coef_.shape) > 1 else model.coef_
            top_idx = np.argsort(np.abs(coefs))[-5:]
            return {X.columns[i]: float(coefs[i]) for i in top_idx}
        elif hasattr(model, "feature_importances_"):
            imps = model.feature_importances_
            top_idx = np.argsort(imps)[-5:]
            return {X.columns[i]: float(imps[i]) for i in top_idx}
    except Exception as e:
        return {"explain_fallback_error": str(e)}
    return {"explain_info": "No explainable features found."}

def client_report(df, client_id, model_path=None):
    df_en = schema.coerce_and_enrich(df)
    cdf = df_en[df_en['unique_client_id'].astype(str)==str(client_id)]
    if cdf.empty:
        return {'error':'Client not found','client_id':client_id}

    rep = {
        'client_id': client_id,
        'towns': list(cdf['town'].unique()) if 'town' in cdf.columns else [],
        'regions': list(cdf['region'].unique()) if 'region' in cdf.columns else [],
        'loans': len(cdf),
        'total_amount': float(cdf['loan_amount_num'].sum()) if 'loan_amount_num' in cdf else None,
        'avg_amount': float(cdf['loan_amount_num'].mean()) if 'loan_amount_num' in cdf else None,
        'status_counts': cdf['loan_status'].value_counts().to_dict() if 'loan_status' in cdf else {}
    }

    if model_path and os.path.exists(model_path):
        try:
            model = joblib.load(model_path)
            X,y,en = schema.prepare_for_ml(cdf)
            if not X.empty:
                prob = model.predict_proba(X)[:,1].mean()
                rep['pred_default_prob'] = float(prob)
                try:
                    expl = utils.explain(model,X,n=min(50,len(X)))
                    rep['shap_preview'] = str(expl)[:500]
                except Exception:
                    rep['shap_preview'] = _fallback_features(model,X)
        except Exception as e:
            rep['pred_error'] = str(e)

    return rep

def export_client_reports(df, model_path=None, out_csv='client_reports.csv'):
    df_en = schema.coerce_and_enrich(df)
    clients = df_en['unique_client_id'].unique()
    rows=[client_report(df_en,cid,model_path) for cid in clients]
    outp = OUT_DIR/out_csv
    pd.DataFrame(rows).to_csv(outp,index=False)
    return str(outp)
"""
(REPORTS_DIR/"__init__.py").write_text("# init\n")
(REPORTS_DIR/"reports.py").write_text(reports_py.strip()+"\n")

# --- Smoke test ---
print("Running smoke test for reports (clean + fallback)...")
try:
    import importlib, warnings
    sys.path.append("/content/loan_app")
    importlib.invalidate_caches()
    warnings.filterwarnings("ignore", category=UserWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=DeprecationWarning)

    ig = importlib.import_module("modules.synth.enhanced_generator")
    schema = importlib.import_module("modules.schema")
    eng = importlib.import_module("modules.ml.engine")
    repmod = importlib.import_module("modules.reports.reports")

    # generate dataset
    df = ig.make_dataset(300, seed=999)
    X,y,en = schema.prepare_for_ml(df)

    # train baseline
    res = eng.train_baseline(X,y,name="report_test")

    # generate sample report
    r = repmod.client_report(df, df['national_id'].iloc[0], res['model_path'])
    print("Sample report:", json.dumps(r,indent=2)[:300])

    # export all
    csvout = repmod.export_client_reports(en, res['model_path'])
    print("Reports CSV saved:", csvout)

    print("Cell 7 smoke test: ✅ OK")
except Exception as e:
    print("Cell 7 smoke test failed:", e)
    raise

In [ ]:
# -------- Cell 8a/9: Admin Tools Core (audit logs, model registry, exports) --------
import os, sys, json, joblib, warnings
from pathlib import Path

BASE = Path("/content/loan_app")
ADMIN_DIR = BASE / "modules" / "admin"
LOG_DIR = BASE / "logs"
ADMIN_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)

admin_py = r"""
import os, json, joblib, datetime
from pathlib import Path
import pandas as pd

LOG_DIR = Path('/content/loan_app/logs')
LOG_DIR.mkdir(parents=True, exist_ok=True)

# ---- Audit Logging ----
def audit_log(event, user="system", meta=None):
    ts = datetime.datetime.utcnow().isoformat()
    log_entry = {"ts": ts, "user": user, "event": event, "meta": meta or {}}
    logf = LOG_DIR/"audit.log"
    with open(logf, "a") as f:
        f.write(json.dumps(log_entry)+"\n")
    return log_entry

# ---- Model Registry ----
MODEL_REGISTRY = LOG_DIR/"model_registry.json"

def register_model(name, model_path, metrics, user="admin"):
    registry = []
    if MODEL_REGISTRY.exists():
        try:
            registry = json.loads(MODEL_REGISTRY.read_text())
        except:
            registry = []
    entry = {"name": name, "model_path": model_path, "metrics": metrics,
             "user": user, "ts": datetime.datetime.utcnow().isoformat()}
    registry.append(entry)
    MODEL_REGISTRY.write_text(json.dumps(registry, indent=2))
    audit_log("register_model", user=user, meta=entry)
    return entry

def list_models():
    if MODEL_REGISTRY.exists():
        return json.loads(MODEL_REGISTRY.read_text())
    return []

def pin_model(name):
    models = list_models()
    if not models: return None
    latest = [m for m in models if m["name"]==name]
    if not latest: return None
    pinned = latest[-1]
    (LOG_DIR/"pinned_model.json").write_text(json.dumps(pinned, indent=2))
    audit_log("pin_model", meta=pinned)
    return pinned

def get_pinned_model():
    p = LOG_DIR/"pinned_model.json"
    if p.exists():
        return json.loads(p.read_text())
    return None

# ---- Data Exports ----
EXPORTS_DIR = LOG_DIR/"exports"
EXPORTS_DIR.mkdir(parents=True, exist_ok=True)

def export_dataset(df, name="dataset_export.csv", fmt="csv"):
    outp = EXPORTS_DIR/name
    if fmt=="csv":
        df.to_csv(outp,index=False)
    elif fmt=="json":
        df.to_json(outp,orient="records")
    audit_log("export_dataset", meta={"file": str(outp), "fmt": fmt})
    return str(outp)
"""
(ADMIN_DIR/"__init__.py").write_text("# init\n")
(ADMIN_DIR/"admin_tools.py").write_text(admin_py.strip()+"\n")

# --- Smoke test ---
print("Running smoke test for admin tools...")
try:
    import importlib
    sys.path.append("/content/loan_app")
    importlib.invalidate_caches()
    adm = importlib.import_module("modules.admin.admin_tools")

    # audit log test
    e = adm.audit_log("smoke_test", user="tester", meta={"note":"admin tools check"})
    print("Audit log entry:", e)

    # fake registry entry
    reg = adm.register_model("demo_model","/tmp/demo.pkl",{"auc":0.75}, user="tester")
    print("Registered model:", reg)

    # pin model
    pin = adm.pin_model("demo_model")
    print("Pinned model:", pin)

    # list models
    print("Model list:", adm.list_models())

    # export dummy dataset
    import pandas as pd
    df = pd.DataFrame({"a":[1,2,3],"b":[4,5,6]})
    exp = adm.export_dataset(df,"demo.csv")
    print("Exported dataset:", exp)

    print("Cell 8a smoke test: ✅ OK")
except Exception as e:
    print("Cell 8a smoke test failed:", e)
    raise

In [ ]:
# -------- Cell 8b/9: Global Analytics & Data Products (super-rich) --------
import os, sys, warnings, json
from pathlib import Path

# ensure repo on path
sys.path.append("/content/loan_app")
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

BASE = Path("/content/loan_app")
ANALYTICS_DIR = BASE / "modules" / "analytics"
EXPORTS_DIR = BASE / "logs" / "exports"
PLOTS_DIR = BASE / "logs" / "plots"
ANALYTICS_DIR.mkdir(parents=True, exist_ok=True)
EXPORTS_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

analytics_py = r"""
# modules.analytics - Global analytics, segments, cohorts, and data products
import os, json, math
import pandas as pd, numpy as np
from pathlib import Path
from modules import schema
from modules.visuals import plot_loan_size_hist, plot_defaults_pie, plot_time_series_by_month
from datetime import datetime

OUT_DIR = Path('/content/loan_app/logs/exports'); OUT_DIR.mkdir(parents=True, exist_ok=True)
PLOT_DIR = Path('/content/loan_app/logs/plots'); PLOT_DIR.mkdir(parents=True, exist_ok=True)

def _safe_load_uploads(limit=None):
    UP = Path('/content/loan_app/data/uploads')
    if not UP.exists(): return pd.DataFrame()
    files = sorted(UP.glob('*'), key=lambda p: p.stat().st_mtime, reverse=True)
    if limit: files = files[:limit]
    dfs=[]
    for p in files:
        try:
            if p.suffix.lower() in ['.csv']: df = pd.read_csv(p)
            else: df = pd.read_excel(p)
            dfs.append(df)
        except Exception:
            continue
    if not dfs: return pd.DataFrame()
    return pd.concat(dfs, ignore_index=True)

def build_master(enforce_enrich=True):
    df = _safe_load_uploads(limit=20)
    if df.empty:
        # fallback: use synth generator if available
        try:
            from modules.synth.enhanced_generator import make_dataset
            df = make_dataset(1000, seed=2024)
        except Exception:
            return pd.DataFrame()
    if enforce_enrich:
        X,y,en = schema.prepare_for_ml(df, aggregate_by_client=False)
        return en
    return df

def compute_topline(df):
    # topline KPIs
    total_loans = len(df)
    total_clients = df['unique_client_id'].nunique() if 'unique_client_id' in df.columns else df['national_id'].nunique()
    defaults = int(df.get('is_default', pd.Series()).sum()) if 'is_default' in df.columns else int(df.get('loan_status',pd.Series()).astype(str).str.contains('default').sum())
    avg_loan = float(df['loan_amount_num'].mean()) if 'loan_amount_num' in df.columns else float(df['loan_amount'].mean())
    median_loan = float(df['loan_amount_num'].median()) if 'loan_amount_num' in df.columns else float(df['loan_amount'].median())
    female_share = float((df.get('gender','').astype(str)=='female').mean()) if 'gender' in df.columns else None
    multi_loan_frac = df.groupby('unique_client_id').size().gt(1).mean() if 'unique_client_id' in df.columns else None
    return {
        'total_loans': int(total_loans),
        'total_clients': int(total_clients),
        'defaults': int(defaults),
        'default_rate': float(defaults / total_loans) if total_loans>0 else None,
        'avg_loan': avg_loan,
        'median_loan': median_loan,
        'female_share': female_share,
        'multi_loan_frac': float(multi_loan_frac) if multi_loan_frac is not None else None
    }

def segment_stats(df, by=['region','town','product','branch','gender']):
    results = {}
    for seg in by:
        if seg not in df.columns: continue
        g = df.groupby(seg).agg(
            loans = ('loan_amount_num','count'),
            avg_amount = ('loan_amount_num','mean'),
            median_amount = ('loan_amount_num','median'),
            defaults = ('is_default','sum'),
        ).reset_index()
        g['default_rate'] = g['defaults'] / g['loans']
        results[seg] = g.sort_values('loans', ascending=False)
    return results

def client_lifetime_metrics(df):
    # per-client aggregates that are attractive to buyers
    grp = df.groupby('unique_client_id').agg(
        loan_count = ('loan_amount_num','count'),
        total_borrowed = ('loan_amount_num','sum'),
        avg_loan = ('loan_amount_num','mean'),
        max_loan = ('loan_amount_num','max'),
        defaults = ('is_default','sum'),
    ).reset_index()
    grp['default_flag'] = (grp['defaults']>0).astype(int)
    # add demographic aggregates (first town/region/gender)
    firsts = df.sort_values('created_date_parsed').groupby('unique_client_id').first().reset_index()
    for c in ['town','region','gender','age_est']:
        if c in firsts.columns:
            grp[c] = grp['unique_client_id'].map(firsts.set_index('unique_client_id')[c].to_dict())
    return grp

def cohort_analysis(df, cohort_period='M'):
    # cohorts by month of first loan
    df['created_date_parsed'] = pd.to_datetime(df['created_date_parsed'], errors='coerce')
    first = df.sort_values('created_date_parsed').groupby('unique_client_id')['created_date_parsed'].min().reset_index()
    first['cohort'] = first['created_date_parsed'].dt.to_period(cohort_period)
    df = df.merge(first[['unique_client_id','cohort']], on='unique_client_id', how='left')
    cohort_table = df.groupby(['cohort','is_default']).size().unstack(fill_value=0)
    return cohort_table

def correlation_and_stats(df):
    # numeric correlations for feature discovery
    numcols = [c for c in df.columns if c.endswith('_num') or c in ['age_est','installment_size','loan_to_income','days_since_issue']]
    if not numcols: return {}
    corr = df[numcols].corr().fillna(0)
    stats = df[numcols].describe().to_dict()
    return {'corr': corr, 'stats': stats}

def top_clients(df, n=50):
    grp = client_lifetime_metrics(df)
    top = grp.sort_values('total_borrowed', ascending=False).head(n)
    return top

def export_data_products(df, prefix='data_product'):
    # export client-level product, segment stats, and raw enriched file
    cli = client_lifetime_metrics(df)
    seg = segment_stats(df)
    now = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    paths = {}
    cli_path = OUT_DIR/f'{prefix}_clients_{now}.csv'; cli.to_csv(cli_path,index=False); paths['clients']=str(cli_path)
    # export segment tables
    for k,v in seg.items():
        p = OUT_DIR/f'{prefix}_segment_{k}_{now}.csv'; v.to_csv(p,index=False); paths[f'segment_{k}']=str(p)
    # export raw enriched
    rawp = OUT_DIR/f'{prefix}_raw_{now}.csv'; df.to_csv(rawp,index=False); paths['raw']=str(rawp)
    return paths

def generate_plots(df):
    p1 = plot_loan_size_hist(df)
    p2 = plot_defaults_pie(df)
    p3 = plot_time_series_by_month(df)
    return [p1,p2,p3]

"""

# write module
(ANALYTICS_DIR / "__init__.py").write_text("# analytics package\n")
(ANALYTICS_DIR / "analytics.py").write_text(analytics_py.strip() + "\n")

# ---------- Smoke Test for analytics ----------
print("Running Cell 8b smoke test (Global Analytics)...")
try:
    import importlib, pandas as pd
    importlib.invalidate_caches()
    sys.path.append("/content/loan_app")
    an = importlib.import_module("modules.analytics.analytics")

    # build master enriched DF (uses uploads or falls back to synth)
    df = an.build_master(enforce_enrich=True)
    if df.empty:
        raise RuntimeError("No data available (uploads missing and synth unavailable)")

    topline = an.compute_topline(df)
    print("Topline KPIs:", json.dumps(topline, indent=2))

    segs = an.segment_stats(df, by=['region','town','product','branch','gender'])
    # print short summary of top 5 regions
    if 'region' in segs:
        print("Top regions by loans:")
        print(segs['region'].head(5).to_string(index=False))

    # client lifetime & top clients
    clients = an.client_lifetime_metrics(df)
    print("Client metrics sample (top 5):")
    print(clients.head(5).to_string(index=False))

    # cohort table (small preview)
    cohort = an.cohort_analysis(df)
    print("Cohort table shape:", cohort.shape)

    # correlation summary
    corr = an.correlation_and_stats(df)
    print("Numeric features discovered:", list(corr.get('stats',{}).keys())[:10])

    # export data products
    paths = an.export_data_products(df, prefix='prod')
    print("Exported data products to:", paths)

    # generate plots
    plots = an.generate_plots(df)
    print("Saved plots:", plots)

    print("Cell 8b smoke test: ✅ OK")
except Exception as e:
    print("Cell 8b smoke test failed:", e)
    raise

In [ ]:

# -------- Cell 9a/10: Streamlit UI (patched with Refresh Link for Admin) --------
import os, sys
from pathlib import Path

BASE = Path("/content/loan_app")
APP_DIR = BASE / "modules" / "streamlit_app"
APP_DIR.mkdir(parents=True, exist_ok=True)

ui_py = r"""
import streamlit as st
import pandas as pd, importlib, sys
sys.path.append('/content/loan_app')

from modules import auth, schema
from modules.reports import reports
from modules.analytics import analytics
from modules.admin import admin_tools
from modules.visuals import plot_loan_size_hist, plot_defaults_pie, plot_time_series_by_month

# Try importing new_link from runner if available
try:
    from __main__ import new_link
except:
    new_link = None

st.set_page_config(page_title="Loan Analytics Portal", layout="wide")

if "user" not in st.session_state:
    st.session_state.user = None
if "role" not in st.session_state:
    st.session_state.role = None

def login():
    st.sidebar.subheader("Login")
    u = st.sidebar.text_input("Username")
    p = st.sidebar.text_input("Password", type="password")
    if st.sidebar.button("Login", use_container_width=True):
        if auth.check_login(u,p):
            st.session_state.user = u
            if u.lower()=="admin": st.session_state.role="admin"
            else: st.session_state.role="client"
            admin_tools.audit_log("login", user=u)
            st.sidebar.success(f"Welcome, {u}")
        else:
            st.sidebar.error("Invalid credentials")

def logout():
    st.session_state.user=None
    st.session_state.role=None

if not st.session_state.user:
    st.title("Loan Analytics Portal")
    st.markdown("Please login to continue.")
    login()
else:
    st.sidebar.write(f"Logged in as: {st.session_state.user}")
    if st.sidebar.button("Logout", use_container_width=True):
        logout()
        st.rerun()

    tabs = ["Dashboard","Reports","Uploads"]
    if st.session_state.role=="admin":
        tabs += ["Admin Sandbox","Global Analytics"]

    choice = st.sidebar.radio("Navigation", tabs)

    if choice=="Dashboard":
        st.title("📊 Loan Dashboard")
        df = analytics.build_master(enforce_enrich=True)
        if df.empty:
            st.warning("No data available. Please upload datasets.")
        else:
            kpis = analytics.compute_topline(df)
            c1,c2,c3,c4 = st.columns(4)
            c1.metric("Total Loans", f"{kpis['total_loans']:,}")
            c2.metric("Clients", f"{kpis['total_clients']:,}")
            c3.metric("Default Rate", f"{kpis['default_rate']*100:.1f}%")
            c4.metric("Female Share", f"{kpis['female_share']*100:.1f}%" if kpis['female_share'] else "-")

            st.subheader("Visuals")
            c1,c2,c3 = st.columns(3)
            with c1: st.image(plot_loan_size_hist(df))
            with c2: st.image(plot_defaults_pie(df))
            with c3: st.image(plot_time_series_by_month(df))

    elif choice=="Uploads":
        st.title("📤 Upload Dataset")
        f = st.file_uploader("Upload CSV", type=["csv"])
        if f is not None:
            upath = BASE/"data"/"uploads"/f.name
            pd.read_csv(f).to_csv(upath, index=False)
            admin_tools.audit_log("upload", user=st.session_state.user, meta={"file":str(upath)})
            st.success(f"Uploaded to {upath}")

    elif choice=="Reports":
        st.title("📑 Client Reports")
        df = analytics.build_master(enforce_enrich=True)
        if df.empty:
            st.warning("No data available.")
        else:
            cid = st.text_input("Enter Client ID")
            if st.button("Generate Report", use_container_width=True):
                rep = reports.client_report(df, cid, model_path=None)
                st.json(rep)
            if st.checkbox("Export All Client Reports"):
                outp = reports.export_client_reports(df, model_path=None)
                st.success(f"Exported to {outp}")

    elif choice=="Admin Sandbox" and st.session_state.role=="admin":
        st.title("🛠️ Admin Sandbox")
        df = analytics.build_master(enforce_enrich=True)
        if st.button("Train Baseline Model", use_container_width=True):
            X,y,en = schema.prepare_for_ml(df)
            eng = importlib.import_module("modules.ml.engine")
            res = eng.train_baseline(X,y,name="sandbox")
            admin_tools.register_model("sandbox", res['model_path'], res['metrics'])
            st.success(f"Model trained. AUC={res['metrics']['auc']:.3f}")
        if st.button("List Models", use_container_width=True):
            st.json(admin_tools.list_models())
        if st.button("Get Pinned Model", use_container_width=True):
            st.json(admin_tools.get_pinned_model())
        if new_link:
            if st.button("🔄 Refresh Portal Link", use_container_width=True):
                url = new_link()
                st.success(f"New link: {url}")

    elif choice=="Global Analytics" and st.session_state.role=="admin":
        st.title("🌍 Global Analytics")
        df = analytics.build_master(enforce_enrich=True)
        if df.empty:
            st.warning("No data.")
        else:
            topline = analytics.compute_topline(df)
            st.subheader("Topline KPIs")
            st.json(topline)
            st.subheader("Segments by Region")
            segs = analytics.segment_stats(df, by=['region'])
            if 'region' in segs:
                st.dataframe(segs['region'])
            st.subheader("Top Clients")
            st.dataframe(analytics.top_clients(df, n=20))
"""

(APP_DIR/"main.py").write_text(ui_py.strip()+"\n")

print("Cell 9a patched: streamlit_app/main.py ✅ (Admin can refresh portal link inside UI)")

In [ ]:

# --- HARDENED PRE-RUNNER (add before your existing runner) ---
import os, sys, time, subprocess, zipfile, signal, psutil
from pathlib import Path
from datetime import datetime

BASE = Path("/content/loan_app")
MODULES = BASE / "modules"
APP_DIR = MODULES / "streamlit_app"
DATA_UPLOADS = BASE / "data" / "uploads"
BACKUP_DIR = Path("/content")
LOG_FILE = Path("/content/streamlit.log")
URL_FILE = Path("/content/LAST_PUBLIC_URL.txt")

# 1. Ensure dirs exist
for d in [BASE, MODULES, APP_DIR, DATA_UPLOADS]:
    d.mkdir(parents=True, exist_ok=True)

# 2. Kill stale processes (Streamlit/ngrok)
import psutil
for proc in psutil.process_iter(attrs=["pid","name","cmdline"]):
    try:
        if any("streamlit" in c for c in proc.info["cmdline"]) or "ngrok" in proc.info["name"].lower():
            print(f"⚠️ Killing stale process {proc.info}")
            proc.kill()
    except Exception:
        pass

# 3. Backup rotation (keep last 3)
def rotate_backups():
    backups = sorted(BACKUP_DIR.glob("LoanIQAppBackup-*.zip"))
    while len(backups) >= 3:
        old = backups.pop(0)
        old.unlink(missing_ok=True)
    ts = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
    new_backup = BACKUP_DIR / f"LoanIQAppBackup-{ts}.zip"
    with zipfile.ZipFile(new_backup, "w") as z:
        for folder in [MODULES]:
            for root, _, files in os.walk(folder):
                for f in files:
                    fp = Path(root) / f
                    z.write(fp, fp.relative_to(BASE))
    print(f"📦 Rotated backup saved: {new_backup}")

if APP_DIR.exists():
    rotate_backups()

# 4. Auto-patch sys.path so 'modules' is always found
if str(BASE) not in sys.path:
    sys.path.insert(0, str(BASE))
print("✅ sys.path patched for modules import")

# 5. Ensure Admin user exists
from modules import auth
auth.init_db()
print("✅ Admin user ensured in DB")

# 6. Save timestamp
print(f"⏰ Startup time: {datetime.utcnow().isoformat()}")

# 7. Prepare log file
if LOG_FILE.exists():
    LOG_FILE.unlink()
open(LOG_FILE, "w").close()
print(f"📝 Logs will be written to {LOG_FILE}")

# 8. Add ngrok retry wrapper (imported later by runner)
from pyngrok import ngrok
def safe_connect(port=8501, retries=2):
    for i in range(retries):
        try:
            tunnel = ngrok.connect(port)
            url = tunnel.public_url
            with open(URL_FILE, "w") as f:
                f.write(url)
            print(f"🌍 Ngrok tunnel established: {url}")
            return tunnel
        except Exception as e:
            print(f"❌ Ngrok failed (attempt {i+1}): {e}")
            time.sleep(3)
            try: ngrok.kill()
            except: pass
    raise RuntimeError("Ngrok failed after retries.")

print("✅ Hardened pre-run checks complete.")

In [ ]:
# --- Repo Doctor: Deep Inspect + Auto-Fix ---
import os, sys, sqlite3, subprocess, signal
from pathlib import Path
import importlib.util

FIX = True  # set True to auto-fix DBs and kill stale processes

BASE = Path("/content/loan_app")
print(f"✅ Base exists: {BASE}")

# 1. Ensure BASE is on sys.path
if str(BASE) not in sys.path:
    sys.path.insert(0, str(BASE))
print("✅ BASE already in sys.path")

# 2. Critical modules check
critical = [
    "modules.auth",
    "modules.schema",
    "modules.synth.generator",
    "modules.ml.engine",
    "modules.ml.utils",
    "modules.ml.audit",
]
for mod in critical:
    try:
        importlib.import_module(mod)
        print(f"✅ Import OK: {mod}")
    except Exception as e:
        print(f"❌ Import FAIL: {mod} -> {e}")

# 3. DB checks + auto-repair
def ensure_table(db_path, ddl, check_sql=None, seed_sql=None, seed_params=None):
    os.makedirs(os.path.dirname(db_path), exist_ok=True)
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    if check_sql:
        try:
            cur.execute(check_sql)
            cur.fetchone()
            print(f"✅ Table OK in {db_path}")
            conn.close()
            return
        except Exception:
            print(f"⚠️ Table missing in {db_path}, repairing...")
    cur.execute(ddl)
    if seed_sql:
        cur.execute(seed_sql, seed_params or ())
    conn.commit()
    conn.close()
    print(f"✅ Table repaired in {db_path}")

if FIX:
    # Users DB
    ensure_table(
        str(BASE/"data"/"users.db"),
        """CREATE TABLE IF NOT EXISTS users (
            username TEXT PRIMARY KEY,
            password TEXT,
            role TEXT
        )""",
        check_sql="SELECT * FROM users LIMIT 1",
        seed_sql="INSERT OR IGNORE INTO users VALUES (?,?,?)",
        seed_params=("admin","admin123","admin")
    )
    # Audit DB
    ensure_table(
        str(BASE/"data"/"audit.db"),
        """CREATE TABLE IF NOT EXISTS audit_log (
            ts TEXT,
            user TEXT,
            action TEXT,
            details TEXT
        )""",
        check_sql="SELECT * FROM audit_log LIMIT 1"
    )

# 4. Kill stale processes
if FIX:
    procs = ["streamlit", "ngrok", "lt"]
    for p in procs:
        try:
            subprocess.run(["pkill","-f",p], check=False)
            print(f"✅ Killed stale {p} processes (if any).")
        except Exception:
            pass

# 5. Streamlit app presence
app_path = BASE/"modules"/"streamlit_app"/"app.py"
if app_path.exists():
    try:
        compile(app_path.read_text(), str(app_path), "exec")
        print(f"✅ Streamlit app found & compiles: {app_path}")
    except Exception as e:
        print(f"❌ Streamlit app compile error: {e}")
else:
    print("❌ Streamlit app missing")

print("🔍 Repo Doctor finished. Ready to run your main runner.")

In [ ]:

# --- FINAL RUNNER WITH AUTO-BACKUP, AUTO-RESTORE & DB AUTO-REPAIR (no tunnel password) ---
import os, sys, time, subprocess, zipfile, signal
from pathlib import Path

# 1. Paths
BASE = Path("/content/loan_app")
APP_PATH = BASE / "modules" / "streamlit_app" / "app.py"
HELPERS_PATH = BASE / "setup_helpers.py"
BACKUP = Path("/content/LoanIQAppBackup.zip")
BASE.mkdir(parents=True, exist_ok=True)

# 2. Install exact versions
!pip install pyngrok==5.2.1 streamlit --quiet

# 3. Kill stale processes
import psutil
for proc in psutil.process_iter(attrs=["pid","name","cmdline"]):
    try:
        cmd = " ".join(proc.info["cmdline"]) if proc.info["cmdline"] else ""
        if "streamlit" in cmd or "ngrok" in cmd or "lt" in cmd:
            os.kill(proc.info["pid"], signal.SIGKILL)
            print(f"⚠️ Killed stale process {proc.info['pid']} ({cmd})")
    except Exception:
        pass
# 3b. Kill broken ngrok configs (they cause "tunnel not found" errors)
for bad in ["/root/.config/ngrok/ngrok.yml", "/root/.ngrok2/ngrok.yml"]:
    try:
        if os.path.exists(bad):
            os.remove(bad)
            print(f"⚠️ Removed old ngrok config at {bad}")
    except Exception as e:
        print(f"⚠️ Could not remove {bad}: {e}")

# Also tell ngrok to ignore configs completely
os.environ["NGROK_CONFIG"] = "null"

# 4. Always rewrite helpers file fresh
helpers_code = r"""
import os, zipfile
from pathlib import Path

BASE = Path("/content/loan_app")

def restore_modules():
    modules_dir = BASE / "modules"
    if modules_dir.exists() and any(modules_dir.glob("**/*.py")):
        print("✅ Modules already exist.")
        return True
    archive = Path("/content/LoanIQAppBackup.zip")
    if archive.exists():
        with zipfile.ZipFile(archive,'r') as z: z.extractall("/content")
        print("✅ Restored modules from backup.")
        return True
    print("❌ No modules or backup found.")
    return False

def restore_app():
    app_path = BASE/"modules"/"streamlit_app"/"app.py"
    if app_path.exists():
        print("✅ app.py exists.")
        return True
    archive = Path("/content/LoanIQAppBackup.zip")
    if archive.exists():
        with zipfile.ZipFile(archive,'r') as z: z.extractall("/content")
        print("✅ Restored app.py from backup.")
        return app_path.exists()
    print("❌ app.py missing and no backup found.")
    return False
"""
HELPERS_PATH.write_text(helpers_code)
print(f"✅ setup_helpers.py refreshed at {HELPERS_PATH}")

# 5. Import helpers
sys.path.insert(0, str(BASE))
import setup_helpers

# 5b. Ensure database schema exists (auto-repair)
import sqlite3

def ensure_db(path, schema_sql):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    conn = sqlite3.connect(path)
    cur = conn.cursor()
    for stmt in schema_sql:
        cur.execute(stmt)
    conn.commit()
    conn.close()
    print(f"✅ DB checked: {path}")

# users.db
ensure_db(str(BASE/"data"/"users.db"), [
    """CREATE TABLE IF NOT EXISTS users (
        username TEXT PRIMARY KEY,
        password TEXT,
        role TEXT,
        created_at TEXT
    )"""
])

# audit.db
ensure_db(str(BASE/"data"/"audit.db"), [
    """CREATE TABLE IF NOT EXISTS audit_log (
        ts TEXT,
        user TEXT,
        action TEXT,
        details TEXT
    )"""
])

# loans.db
ensure_db(str(BASE/"data"/"loans.db"), [
    """CREATE TABLE IF NOT EXISTS clients (
        client_id TEXT PRIMARY KEY,
        name TEXT,
        gender TEXT,
        region TEXT,
        age INTEGER,
        joined_on TEXT
    )""",
    """CREATE TABLE IF NOT EXISTS loans (
        loan_id TEXT PRIMARY KEY,
        client_id TEXT,
        amount REAL,
        status TEXT,
        issued_on TEXT,
        due_on TEXT,
        FOREIGN KEY(client_id) REFERENCES clients(client_id)
    )""",
    """CREATE TABLE IF NOT EXISTS models (
        name TEXT PRIMARY KEY,
        model_path TEXT,
        auc REAL,
        trained_on TEXT
    )"""
])

# Seed default admin if missing
conn = sqlite3.connect(str(BASE/"data"/"users.db"))
cur = conn.cursor()
cur.execute("PRAGMA table_info(users)")
cols = [row[1] for row in cur.fetchall()]  # get column names

cur.execute("SELECT 1 FROM users WHERE username='admin'")
if not cur.fetchone():
    placeholders = ",".join(["?"] * len(cols))
    values = []
    for c in cols:
        if c == "username":
            values.append("admin")
        elif c == "password":
            values.append("admin123")
        elif c == "role":
            values.append("admin")
        else:
            values.append(None)  # default for any extra column
    cur.execute(f"INSERT INTO users ({','.join(cols)}) VALUES ({placeholders})", tuple(values))
    print(f"✅ Default admin user created with columns {cols}")

conn.commit()
conn.close()

# 6. Restore modules & app if needed
ok_mods = setup_helpers.restore_modules()
ok_app  = setup_helpers.restore_app()

# 7. AUTO-BACKUP current state
if APP_PATH.exists():
    with zipfile.ZipFile(BACKUP, "w") as z:
        for folder in [BASE / "modules"]:
            for root, _, files in os.walk(folder):
                for f in files:
                    fp = Path(root) / f
                    z.write(fp, fp.relative_to(BASE))
    print(f"📦 Backup updated at {BACKUP}")
else:
    print("⚠️ Skipping backup: app.py not found yet.")

# Replace the pyngrok section with this:

print("🚀 Launching Streamlit app...")
streamlit_proc = subprocess.Popen(
    ["streamlit", "run", str(APP_PATH), "--server.port=8501", "--server.headless=true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# Give streamlit a few seconds to boot
time.sleep(6)

print("🌍 Creating localtunnel...")
lt_proc = subprocess.Popen(
    ["/tools/node/bin/lt", "--port", "8501", "--subdomain", "loaniqdemo"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
)
# Read the tunnel URL from lt stdout
for line in lt_proc.stdout:
    if "your url is:" in line.lower():
        url = line.split()[-1].strip()
        print(f"🌍 Your app is live at: {url}\n")
        break